# Proposal to Enable Fast TPN Calculations

In this notebook solution direction is explored to enable Total Parenteral Nutrition calculations, like:

tot_fluid = prot_fluid + sodium_fluid + potassium_fluid + gluc_fluid

Each of those fluids can vary between, say, 0.1–500 mL, so each one easily has 100+ possible values. Combine two or more, and you quickly get a quadratic explosion in combinations.

But in practice, we usually want answers to much vaguer questions — like “raise the drip by 10%.” It turns out this fits really nicely with how GenSolver can just calculate min and max values. For discrete variables (like fluid volumes or drip rates), it’s straightforward to set a value as a percentage of the total range of options.

In [1]:
// set up the environment
#load "load.fsx"

open System
open MathNet.Numerics
open Informedica.Utils.Lib.BCL
open Informedica.GenCore.Lib.Ranges
open Informedica.GenForm.Lib
open Informedica.GenUnits.Lib
open Informedica.GenOrder.Lib

Environment.SetEnvironmentVariable("GENPRES_DEBUG", "1")
Environment.SetEnvironmentVariable("GENPRES_PROD", "1")
Environment.SetEnvironmentVariable("GENPRES_URL_ID", "1xhFPiF-e5rMkk7BRSfbOF-XGACeHInWobxRbjYU0_w4")


let print sl = sl |> List.iter (printfn "%s")

// optional use of a logger to dump all calculations
// to the console
let logger = OrderLogging.createConsoleLogger ()


Some helper functions to enable "cherry picking" the application of constraints on the TPN medication types.

In [2]:
// setting up some basic tpn constraints
let tpnConstraints =
    [
        OrderAdjust OrderVariable.Quantity.applyConstraints

        ScheduleFrequency OrderVariable.Frequency.applyConstraints
        ScheduleTime OrderVariable.Time.applyConstraints

        OrderableQuantity OrderVariable.Quantity.applyConstraints
        OrderableDoseCount OrderVariable.Count.applyConstraints
        OrderableDose Order.Orderable.Dose.applyConstraints

        ComponentOrderableQuantity ("", OrderVariable.Quantity.applyConstraints)

        ItemComponentConcentration ("", "", OrderVariable.Concentration.applyConstraints)
        ItemOrderableConcentration ("", "", OrderVariable.Concentration.applyConstraints)
    ]


// apply a change in the order property
let applyPropChange msg propChange ord =
    printfn $"=== Apply PropChange {msg} ==="
    let ord =
        ord
        |> Order.OrderPropertyChange.proc propChange
    ord
    |> Order.solveMinMax true Logging.noOp
    |> function
        | Ok ord -> ord
        | _ ->
            printfn $"=== ERROR {msg} ==="
            ord
    |> fun ord ->
        ord
        |> Order.printTable ConsoleTables.Format.Minimal

        ord


// run a tpn scenario with specific
// percentages for protein, potassium, sodium and glucose
let run 
    proteinPerc 
    potassiumPerc 
    sodiumPerc 
    glucPerc 
    tpn =
    
    tpn
    |> Medication.toOrderDto
    |> Order.Dto.fromDto
    |> Result.map (fun ord ->
        let ord =
            ord
            |> Order.OrderPropertyChange.proc tpnConstraints
    //        |> Order.applyConstraints

        ord
        |> Order.printTable ConsoleTables.Format.Minimal

        let ord =
            ord
            |> Order.solveMinMax true Logging.noOp //logger
            //|> Result.bind (Order.solveMinMax true logger)

        ord
        |> Result.iter (Order.printTable ConsoleTables.Format.Minimal)

        let ord =
            ord
            |> Result.map (fun ord ->
                ord
                |> applyPropChange
                    "Samenstelling C"
                    [
                        ComponentOrderableQuantity ("Samenstelling C", OrderVariable.Quantity.setPercValue proteinPerc)
                    ]
            )

        let ord =
            ord
            |> Result.map (fun ord ->
                ord
                |> applyPropChange
                    "KCl 7,4%"
                    [
                        ComponentOrderableQuantity ("KCl 7,4%", OrderVariable.Quantity.setPercValue potassiumPerc)
                    ]
            )

        let ord =
            ord
            |> Result.map (fun ord ->
                ord
                |> applyPropChange
                    "NaCl 3%"
                    [
                        ComponentOrderableQuantity ("NaCl 3%", OrderVariable.Quantity.setPercValue sodiumPerc)
                    ]
            )

        let ord =
            ord
            |> Result.map (fun ord ->
                ord
                |> applyPropChange
                    "gluc 10%"
                    [
                        ComponentOrderableQuantity ("gluc 10%", OrderVariable.Quantity.setPercValue glucPerc)
                    ]
            )

        ord
    )



Define 2 types of TPN solutions, a simple one and a more complex one.

In [3]:
// "simple TPN solution"
let tpnSimple =
    { Medication.order with
        Id = "f1adf475-919b-4b7d-9e26-6cc502b88e42"
        Name = "samenstelling c"
        Route = "INTRAVENEUS"
        OrderType = TimedOrder
        Adjust =
            11N
            |> ValueUnit.singleWithUnit Units.Weight.kiloGram
            |> Some
        AdjustUnit = Units.Weight.kiloGram |> Some
        Frequencies =
            1N
            |> ValueUnit.singleWithUnit (Units.Count.times |> Units.per Units.Time.day)
            |> Some
        Time =
            { MinMax.empty with
                Min =
                    20N
                    |> ValueUnit.singleWithUnit Units.Time.hour
                    |> Limit.inclusive
                    |> Some
                Max =
                    24N
                    |> ValueUnit.singleWithUnit Units.Time.hour
                    |> Limit.inclusive
                    |> Some
            }
        Dose =
            { DoseLimit.limit with
                DoseLimitTarget = "vloeistof" |> LimitTarget.ShapeLimitTarget
                AdjustUnit = Units.Weight.kiloGram |> Some
                QuantityAdjust =
                    { MinMax.empty with
                        Max =
                            (755N / 10N)
                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                            |> Limit.inclusive
                            |> Some
                    }
            }
            |> Some
        DoseCount =
            { MinMax.empty with
                Min = 1N |> ValueUnit.singleWithUnit Units.Count.times |> Limit.inclusive |> Some
                Max = 1N |> ValueUnit.singleWithUnit Units.Count.times |> Limit.inclusive |> Some
            }
        Components =
            [
                // Samenstelling C component
                {
                    Medication.productComponent with
                        Name = "Samenstelling C"
                        Shape = "vloeistof"
                        Quantities =
                            1N
                            |> ValueUnit.singleWithUnit Units.Volume.milliLiter
                            |> Some
                        Divisible = Some (1N)
                        Dose =
                            { DoseLimit.limit with
                                DoseLimitTarget = "Samenstelling C" |> LimitTarget.ComponentLimitTarget
                                AdjustUnit = Units.Weight.kiloGram |> Some
                                QuantityAdjust =
                                    { MinMax.empty with
                                        Min =
                                            10N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                        Max =
                                            25N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                    }
                            }
                            |> Some
                        Substances =
                            [
                                {
                                    Medication.substanceItem with
                                        Name = "eiwit"
                                        Concentrations =
                                            (8N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Mass.gram |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "eiwit" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 100N)
                                                            |> ValueUnit.singleWithUnit (Units.Mass.gram |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "natrium"
                                        Concentrations =
                                            (1N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "natrium" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 10N)
                                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "kalium"
                                        Concentrations =
                                            (2N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "kalium" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 10N)
                                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                            ]
                }
                // NaCl 3% component
                {
                    Medication.productComponent with
                        Name = "NaCl 3%"
                        Shape = "vloeistof"
                        Quantities =
                            1N
                            |> ValueUnit.singleWithUnit Units.Volume.milliLiter
                            |> Some
                        Divisible = Some (1N)
                        Dose =
                            { DoseLimit.limit with
                                DoseLimitTarget = "NaCl 3%" |> LimitTarget.ComponentLimitTarget
                                AdjustUnit = Units.Weight.kiloGram |> Some
                                QuantityAdjust =
                                    { MinMax.empty with
                                        Min =
                                            6N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                        Max =
                                            6N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                    }
                            }
                            |> Some
                        Substances =
                            [
                                {
                                    Medication.substanceItem with
                                        Name = "natrium"
                                        Concentrations =
                                            (5N / 10N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "natrium" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 10N)
                                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                            ]
                }
                // KCl 7,4% component
                {
                    Medication.productComponent with
                        Name = "KCl 7,4%"
                        Shape = "vloeistof"
                        Quantities =
                            1N
                            |> ValueUnit.singleWithUnit Units.Volume.milliLiter
                            |> Some
                        Divisible = Some (1N)
                        Dose =
                            { DoseLimit.limit with
                                DoseLimitTarget = "KCl 7,4%" |> LimitTarget.ComponentLimitTarget
                                AdjustUnit = Units.Weight.kiloGram |> Some
                                QuantityAdjust =
                                    { MinMax.empty with
                                        Min =
                                            2N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                        Max =
                                            2N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                    }
                            }
                            |> Some
                        Substances =
                            [
                                {
                                    Medication.substanceItem with
                                        Name = "kalium"
                                        Concentrations =
                                            1N
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "kalium" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 10N)
                                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                            ]
                }
                // gluc 10% component
                {
                    Medication.productComponent with
                        Name = "gluc 10%"
                        Shape = "vloeistof"
                        Quantities =
                            1N
                            |> ValueUnit.singleWithUnit Units.Volume.milliLiter
                            |> Some
                        Divisible = Some (1N)
                        Substances =
                            [
                                {
                                    Medication.substanceItem with
                                        Name = "koolhydraat"
                                        Concentrations =
                                            (1N / 10N)
                                            |> ValueUnit.singleWithUnit (Units.Mass.gram |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                            ]
                }
            ]
    }

// print out the tpn
tpnSimple
|> Medication.toString
|> print

And a complex TPN setup

In [4]:
// a complex TPN setup
let tpnComplete =
    { Medication.order with
        Id = "f1adf475-919b-4b7d-9e26-6cc502b88e42"
        Name = "samenstelling c"
        Route = "INTRAVENEUS"
        OrderType = TimedOrder
        Adjust =
            11N
            |> ValueUnit.singleWithUnit Units.Weight.kiloGram
            |> Some
        AdjustUnit = Units.Weight.kiloGram |> Some
        Frequencies =
            1N
            |> ValueUnit.singleWithUnit (Units.Count.times |> Units.per Units.Time.day)
            |> Some
        Time =
            { MinMax.empty with
                Min =
                    20N
                    |> ValueUnit.singleWithUnit Units.Time.hour
                    |> Limit.inclusive
                    |> Some
                Max =
                    24N
                    |> ValueUnit.singleWithUnit Units.Time.hour
                    |> Limit.inclusive
                    |> Some
            }
        Dose =
            { DoseLimit.limit with
                DoseLimitTarget = "vloeistof" |> LimitTarget.ShapeLimitTarget
                AdjustUnit = Units.Weight.kiloGram |> Some
                QuantityAdjust =
                    { MinMax.empty with
                        Max =
                            (755N / 10N)
                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                            |> Limit.inclusive
                            |> Some
                    }
            }
            |> Some
        DoseCount =
            { MinMax.empty with
                Min = 1N |> ValueUnit.singleWithUnit Units.Count.times |> Limit.inclusive |> Some
                Max = 1N |> ValueUnit.singleWithUnit Units.Count.times |> Limit.inclusive |> Some
            }
        Components =
            [
                // Samenstelling C component
                {
                    Medication.productComponent with
                        Name = "Samenstelling C"
                        Shape = "vloeistof"
                        Quantities =
                            1N
                            |> ValueUnit.singleWithUnit Units.Volume.milliLiter
                            |> Some
                        Divisible = Some (1N)
                        Dose =
                            { DoseLimit.limit with
                                DoseLimitTarget = "Samenstelling C" |> LimitTarget.ComponentLimitTarget
                                AdjustUnit = Units.Weight.kiloGram |> Some
                                QuantityAdjust =
                                    { MinMax.empty with
                                        Min =
                                            10N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                        Max =
                                            25N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                    }
                            }
                            |> Some
                        Substances =
                            [
                                {
                                    Medication.substanceItem with
                                        Name = "energie"
                                        Concentrations =
                                            (32N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Energy.kiloCalorie |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "eiwit"
                                        Concentrations =
                                            (8N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Mass.gram |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "eiwit" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 100N)
                                                            |> ValueUnit.singleWithUnit (Units.Mass.gram |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "natrium"
                                        Concentrations =
                                            (1N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "natrium" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 10N)
                                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "kalium"
                                        Concentrations =
                                            (2N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "kalium" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 10N)
                                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "calcium"
                                        Concentrations =
                                            (3N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "fosfaat"
                                        Concentrations =
                                            (2N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "magnesium"
                                        Concentrations =
                                            (1N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "chloor"
                                        Concentrations =
                                            (7N / 100N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                            ]
                }
                // NaCl 3% component
                {
                    Medication.productComponent with
                        Name = "NaCl 3%"
                        Shape = "vloeistof"
                        Quantities =
                            1N
                            |> ValueUnit.singleWithUnit Units.Volume.milliLiter
                            |> Some
                        Divisible = Some (1N)
                        Dose =
                            { DoseLimit.limit with
                                DoseLimitTarget = "NaCl 3%" |> LimitTarget.ComponentLimitTarget
                                AdjustUnit = Units.Weight.kiloGram |> Some
                                QuantityAdjust =
                                    { MinMax.empty with
                                        Min =
                                            6N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                        Max =
                                            6N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                    }
                            }
                            |> Some
                        Substances =
                            [
                                {
                                    Medication.substanceItem with
                                        Name = "natrium"
                                        Concentrations =
                                            (5N / 10N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "natrium" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 10N)
                                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "chloor"
                                        Concentrations =
                                            (5N / 10N)
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                            ]
                }
                // KCl 7,4% component
                {
                    Medication.productComponent with
                        Name = "KCl 7,4%"
                        Shape = "vloeistof"
                        Quantities =
                            1N
                            |> ValueUnit.singleWithUnit Units.Volume.milliLiter
                            |> Some
                        Divisible = Some (1N)
                        Dose =
                            { DoseLimit.limit with
                                DoseLimitTarget = "KCl 7,4%" |> LimitTarget.ComponentLimitTarget
                                AdjustUnit = Units.Weight.kiloGram |> Some
                                QuantityAdjust =
                                    { MinMax.empty with
                                        Min =
                                            2N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                        Max =
                                            2N
                                            |> ValueUnit.singleWithUnit (Units.Volume.milliLiter |> Units.per Units.Weight.kiloGram)
                                            |> Limit.inclusive
                                            |> Some
                                    }
                            }
                            |> Some
                        Substances =
                            [
                                {
                                    Medication.substanceItem with
                                        Name = "kalium"
                                        Concentrations =
                                            1N
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                        Solution =
                                            { SolutionLimit.limit with
                                                SolutionLimitTarget = "kalium" |> LimitTarget.SubstanceLimitTarget
                                                Concentration =
                                                    { MinMax.empty with
                                                        Max =
                                                            (5N / 10N)
                                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                                            |> Limit.inclusive
                                                            |> Some
                                                    }
                                            }
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "chloor"
                                        Concentrations =
                                            1N
                                            |> ValueUnit.singleWithUnit (Units.Molar.milliMole |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                            ]
                }
                // gluc 10% component
                {
                    Medication.productComponent with
                        Name = "gluc 10%"
                        Shape = "vloeistof"
                        Quantities =
                            1N
                            |> ValueUnit.singleWithUnit Units.Volume.milliLiter
                            |> Some
                        Divisible = Some (1N)
                        Substances =
                            [
                                {
                                    Medication.substanceItem with
                                        Name = "energie"
                                        Concentrations =
                                            (4N / 10N)
                                            |> ValueUnit.singleWithUnit (Units.Energy.kiloCalorie |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                                {
                                    Medication.substanceItem with
                                        Name = "koolhydraat"
                                        Concentrations =
                                            (1N / 10N)
                                            |> ValueUnit.singleWithUnit (Units.Mass.gram |> Units.per Units.Volume.milliLiter)
                                            |> Some
                                }
                            ]
                }
            ]
    }

// print out the more complex (complete) tpn
tpnComplete
|> Medication.toString
|> print


And we can run a specific scenario with setting protein to 50%, potassium to 10%, sodium to 5% and glucose to 100%

In [5]:
tpnSimple
|> run 50 10 5 100 
|> ignore


The same for a more complex (complete) TPN medication order

In [6]:
tpnComplete
|> run 70 10 5 1
|> ignore